### Import model

In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd
import matplotlib.pyplot as plt
model = read_sbml_model('iML1515.xml')

### Create metabolites

In [2]:
ncam_e=Metabolite('ncam_e', name='nicotineamide', compartment='e', formula = 'C6H6N2O'  ),
na_c=Metabolite('na_c', name='Nicotinic acid',  compartment='c', formula = 'C6H5NO2' ),
namn_c=Metabolite('namn_c', name='nicotinic acid mononucleotide',  compartment='c', formula = 'C11H14NO9P' ),
nr_c=Metabolite('nr_c', name='nicotineamide riboside',  compartment='c', formula = 'C11H15N2O5' ),
nr_e=Metabolite('nr_e', name='nicotineamide riboside',  compartment='e',formula = 'C11H15N2O5' ),
na_e=Metabolite('na_e', name='Nicotinic acid',  compartment='e', formula = 'C6H5NO2' ),
nh3_c=Metabolite('nh3_c', name='Ammonia',  compartment='c', formula = 'NH3' ),
mets=[ncam_e,na_c,namn_c,nr_c,nr_e,na_e,nh3_c]
for i in range(7):
    model.add_metabolites(mets[i])

In [3]:
# NA Pathway

# NA <=> NAMN https://www.kegg.jp/entry/6.3.4.21
nico_acid_phosp_trans = Reaction ('PNCB')
nico_acid_phosp_trans.name = 'nicotinic acid phosphoribosyl transferase'

nico_acid_phosp_trans.add_metabolites  (({model.metabolites.na_c: -1,
                                          model.metabolites.h2o_c: -1,
                                          model.metabolites.prpp_c: -1,
                                          model.metabolites.ppi_c: 1,
                                          model.metabolites.adp_c: 1,
                                          model.metabolites.pi_c: 1,
                                          model.metabolites.namn_c: 1})) 
#NAMN <=> NMN https://www.kegg.jp/entry/6.3.1.5
nico_syntase= Reaction ('FtNadE')
nico_syntase.name = 'nicotineamide mononucleotide synthetase'
                                        
nico_syntase.add_metabolites (({model.metabolites.namn_c:-1,
                                model.metabolites.atp_c:-1,
                                model.metabolites.nh3_c:-1,
                                model.metabolites.amp_c:1,
                                model.metabolites.ppi_c:1,
                                model.metabolites.nmn_c: 1})) 
#EX_NA -> NA https://tcdb.org/search/result.php?tc=9.A.23
nicotinic_acid_trans = Reaction('NiaP2')
nicotinic_acid_trans.name = 'nicotinic acid transporter'

nicotinic_acid_trans.add_metabolites (({model.metabolites.na_c: 1, 
                                  model.metabolites.na_e: -1}))

# Transport reaction cytosol -> extracellular 

nmn_ctoe = Reaction('BMpnuC')
nmn_ctoe.name = 'NMN transporter'
nmn_ctoe.add_metabolites({ model.metabolites.nmn_c: -1,
                          model.metabolites.nmn_e: 1})


In [4]:
# Set up pathway
na_path = [nico_acid_phosp_trans,nico_syntase,nicotinic_acid_trans,nmn_ctoe ]
model.add_reactions(na_path)
model.reactions.PNCB.lower_bound=-10
model.reactions.FtNadE.lower_bound=-10
model.reactions.NiaP2.lower_bound=-10
model.reactions.BMpnuC.lower_bound=-10
model.reactions.PNCB.reversibility= True
model.reactions.FtNadE.reversibility= True
model.reactions.NiaP2.reversibility= True
model.reactions.BMpnuC.reversibility= True
model.reactions.NMNDA.knock_out()

C:\Users\Aljosha\anaconda3\lib\site-packages\cobra\core\reaction.py:810: UserWarning: Setting reaction reversibility is ignored
  warn("Setting reaction reversibility is ignored")


In [5]:
model.add_boundary(model.metabolites.get_by_id("ncam_e"), type="exchange")
model.add_boundary(model.metabolites.get_by_id("nr_e"), type="exchange")
model.add_boundary(model.metabolites.get_by_id("na_e"), type="exchange")
model.reactions.EX_ncam_e.lower_bound=-10
model.reactions.EX_nr_e.lower_bound=-10
model.reactions.EX_na_e.lower_bound=-10
medium=model.medium
medium['EX_ncam_e'] = 10
medium['EX_nr_e'] = 10
medium["EX_na_e"] = 10
medium["EX_glc__D_e"] = 10
model.medium=medium

In [6]:
with model:
    growth = model.optimize()
    max_growth_rate = growth.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
    print('NA-pathway biomass growth', max_growth_rate,'mmol/gDW*h')

NA-pathway biomass growth 0.8769972144269806 mmol/gDW*h


In [7]:
with model:
    print('With NA pathway:')
    model.objective = model.reactions.EX_nmn_e #Setting the objective to the reaction, which is producing NMN
    nmn_production = model.optimize().objective_value
    print('Maximum productivity of NMN =', nmn_production, 'mmol/gDW*h')

    glc_uptake_flux = model.reactions.EX_glc__D_e.flux 
    max_yield = nmn_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-NMN/mmol-glc') 

With NA pathway:
Maximum productivity of NMN = 4.473345935727788 mmol/gDW*h
Maximum theoretical yield = 0.44733459357277877 mmol-NMN/mmol-glc


In [8]:
# Run double optimization, both biomass and product.
solution=model.optimize()

with model:
    
    # Objective set to NMN
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound=(solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])/2
    model.objective = model.reactions.EX_nmn_e
    solution = model.optimize()
    print('Optimized for 1/2 of max. Biomass')
    print('NMN production:', solution.fluxes['EX_nmn_e'],'[mmol gDW^-1 h^-1]')
    print('Biomass growth',solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'],'[mmol gDW^-1 h^-1]')

Optimized for 1/2 of max. Biomass
NMN production: 2.242050599929578 [mmol gDW^-1 h^-1]
Biomass growth 0.43849860721348133 [mmol gDW^-1 h^-1]
